<a href="https://colab.research.google.com/github/shizoda/education/blob/main/machine_learning/basics/spam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ベイズの定理を使ったスパムフィルタリング

ここではベイズの定理を用いて、届いたメールがスパムか否かを判定する AI をつくってみましょう。AI という語はあまり技術的な語ではありませんが、最新の AI もこのような考え方に基づいていますので、ここでは敢えて AI という語を使っています。

### 🤔 「ベイズの定理」とは

ベイズの定理は、一言でいうと**「自分の『思い込み』を、新しいデータ（事実）で更新していく方法」**です。

例えば、あなたが「Aさんは時間にルーズだ」という思い込み（最初の確率）を持っていたとします。でも、最近Aさんが5回連続で約束の時間ぴったりに来た、という新しいデータを観測したらどうでしょう？

「あれ、もしかしてAさんは実は真面目なのかも…？」と、思い込みが更新されますね。この「思い込みの更新」を、数学的にきっちり計算できるようにしたのがベイズの定理です。

$$ P(H∣D)=P(D)P(D∣H)⋅P(H) $$ ​

この式は、まさにその考え方を表しています。

- 事前確率 $ P(H)$ ：最初の思い込み（Aさんはルーズだ！）
- 尤度 $ P(D∣H) $：もし思い込みが正しかったら、このデータ（5回連続で時間通り）が起こる確からしさ
- 事後確率 $ P(H∣D) $：データを踏まえた後の、更新された思い込み（やっぱりAさんは真面目かも！）

### 🤖機械学習への応用

そして、この「データで思い込みを更新する」仕組みをコンピューターにやらせるのが、機械学習のひとつの考え方です。今回のテーマである「ナイーブベイズ」によるスパムフィルターも、まさにこの仕組みを使っています。

- 最初の思い込み：「『当選』『無料』みたいな単語が入ってるメールは、たぶんスパムだろうな〜」
- データで学習：たくさんのスパムメールと普通のメール（教師データ）を AI に読み込ませる。
- 思い込みの更新：AIはデータから、「『当選』という単語は、普通のメールよりスパムメールに XX 倍多く出現するな」といったことを学習します。

こうして、新しいメールが届くたびに、AI は学習済みの知識を使って「このメールはスパムらしいか、らしくないか」を判断できるのです。それでは、実際にPythonのコードを見ながら、このスパムフィルターがどう作られるのか体験していきましょう。

### ナイーブベイズ

今回使う「ナイーブベイズ」という手法の「ナイーブ（Naive）」には、「単純な、うぶな」といった意味があります。

メールに含まれる単語たちの関係性を**「あえて単純に（ナイーブに）考える」**ということが意味です。本当は「当選」と「賞金」のように、一緒に出てきやすい単語の組み合わせはあります。でも、その複雑な関係を全部計算しようとすると、ものすごく大変です。そこでナイーブベイズは、「それぞれの単語は、他の単語とは関係なく独立して登場する」と割り切って、シンプルに計算します。

### スパムフィルタにおけるベイズの定理の適用
スパムフィルタリングにおいて、次のように解釈します：
- 仮説 $ H $：メールがスパムであること
- データ $ D $：メールに特定の単語が含まれていること

これを用いて、メールがスパムである確率 $ P(\text{spam}|\text{email}) $ を計算します。

## ステップ1: 教師データを用意 📧

まずは、AIにとっての「教師」となるデータ（教師データ）を準備します。何が「スパム」で、何が「スパムじゃない（ハム）」なのかを教えるための、お手本メール集です。メールの文面（データ）と、それがスパムかハムか（正解）のペアが多数あります。

In [ ]:
import pandas as pd

# サンプルデータセットの作成
data = {
    'email': [
        'Win a million dollars now',   # スパム
        'Lowest prices in pharmacy',    # スパム
        'Hello, how are you?',          # 非スパム
        'Meeting tomorrow at 10am',     # 非スパム
        'Congratulations, you won a prize', # スパム
        'Get cheap meds online',        # スパム
        'Can we reschedule our appointment?', # 非スパム
        'Your account has been hacked'  # スパム
    ],
    'label': ['spam', 'spam', 'ham', 'ham', 'spam', 'spam', 'ham', 'spam']
}

df = pd.DataFrame(data)
df


ここでは、8通の短いメールと、それぞれが「spam」か「ham」かという正解ラベルを用意しました。これをもとに学習を進めていきます。

#### 📝課題 1

あなたが普段受け取るメールやメッセージで、「これはスパムっぽいな」と感じるものに含まれがちな単語を 3 個あげてください。日本語でも英語でも構いません。

（回答欄）

## ステップ2: データの前処理

次に、メールのテキストを前処理し、単語の出現回数をカウントします。

各行は１通のメールに相当します。横軸の大半（label列を除く）は単語で、存在回数を表します。label列は、そのメールがスパムか非スパムかを表します。

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizerインスタンスを作成
vectorizer = CountVectorizer()

# メールテキストをフィットし、変換する
X = vectorizer.fit_transform(df['email'])

# 特徴量（単語）の名前を取得 (修正点)
feature_names = vectorizer.get_feature_names_out()

# 単語の出現回数をデータフレームに変換
word_count_df = pd.DataFrame(X.toarray(), columns=feature_names)

# データフレームにラベル列を追加
word_count_df['label'] = df['label']

# 結果のデータフレームを表示
word_count_df

実行すると、巨大な表ができますね。各行が1通のメール、各列がメールに出てきた全単語を表しています。そして、数字はそのメールでその単語が何回使われたか、という回数です。

これで、コンピューターが計算できる形のデータが整いました！

### 📝 課題

もし、新しく「congratulations you won」というメールが来て、それがスパムだったとしたら、上の表はどのようになるでしょうか？

（回答欄）

## ステップ3: 🎩 いよいよベイズの定理でスパム確率を計算

さあ、いよいよベイズの定理を使って、スパムかどうかを判定する仕組みを作っていきます。

まずは、ステップ2で作った単語の数カウンターを使って、「スパムメール」と「ハムメール」それぞれで、各単語がどれくらいの頻度で現れるかを計算します。

これが、新しいメールを判定するための**「証拠」**になります。例えば、「win」や「prize」という単語は、スパムメールによく出てきそうですよね。逆に「meeting」や「reschedule」は、ハムメールで使われそうです。

In [ ]:
import numpy as np

# スパムと非スパムの頻度を計算
spam_emails = word_count_df[word_count_df['label'] == 'spam']
ham_emails = word_count_df[word_count_df['label'] == 'ham']

# 各単語のスパムおよび非スパムの確率を計算
spam_word_probs = (spam_emails.drop('label', axis=1).sum() + 1) / (spam_emails.drop('label', axis=1).sum().sum() + len(feature_names))
ham_word_probs = (ham_emails.drop('label', axis=1).sum() + 1) / (ham_emails.drop('label', axis=1).sum().sum() + len(feature_names))

# 例示用。スパムらしい単語とスパムらしくない単語を挙げています
selected_words = ['win', 'prize', 'pharmacy', 'meeting', 'reschedule']
for word in selected_words:
    spam_count = spam_emails[word].sum()
    ham_count = ham_emails[word].sum()
    spam_prob = spam_word_probs[word]
    ham_prob = ham_word_probs[word]

    print(f"Word: '{word}'")
    print(f"  Spam count: {spam_count}, Ham count: {ham_count}")
    print(f"  P(word|spam): {spam_prob:.4f}, P(word|ham): {ham_prob:.4f}")

$P(\textrm{word}|\textrm{spam})$ は、「そのメールがスパムだと分かっている場合に、その単語が含まれている確率」です。winはスパムメールでの確率が高い（スパムっぽい証拠）、meetingはハムメールでの確率が高い（ハムっぽい証拠）ことが分かりますね。

#### 事前確率の計算

事前確率 $P(\text{spam})$ と $P(\text{ham})$ は、スパムメールおよび非スパムメールの全体に対する割合です。

$$ P(\text{spam}) = \frac{\text{スパムメールの数}}{\text{全メールの数}} $$
$$ P(\text{ham}) = \frac{\text{ハムメールの数}}{\text{全メールの数}} $$

📝 課題

上記の式を Python コードとして記述し、スパムとハムの事前確率をそれぞれ計算してみましょう。

In [ ]:
n_spam  = len(spam_emails)    # スパムメールの数
n_ham   = len(ham_emails)      # 非スパムメールの数
n_total = len(word_count_df) # 全メールの数

P_spam =                 # ←スパムの事前確率の計算
P_ham  =                 # ←　ハムの事前確率の計算
print("スパムの事前確率　", P_spam)
print("非スパムの事前確率", P_ham)

### 推論：新しいメールを判定してみよう

それでは、学習した結果を使って、新しいメールを判定する関数を作ります。

この関数の中では、ベイズの定理を使って、以下の計算をしています。

- __事前確率__（最初の思い込み）を設定

 - log_P_spam: そもそもスパムメールは全体のどれくらいの割合か？
 - log_P_ham: そもそもハムメールは全体のどれくらいの割合か？

- __尤度（証拠）__を掛け合わせる:

新しいメールに含まれる単語を一つずつチェックします。

その単語の「スパムっぽさ（P(word|spam)）」と「ハムっぽさ（P(word|ham)）」を、1.の確率にどんどん掛け合わせていきます。（※実際には、計算を簡単にするために対数を使って足し算します）

- __事後確率__ を比較:
最終的に計算された「総合的なスパムっぽさ」と「総合的なハムっぽさ」を比較します。
スパムっぽさのスコアが高ければ「スパム」、ハムっぽさのスコアが高ければ「非スパム（ハム）」と判定します。



### 数式での説明（難しいと感じる方はスキップしてください）

#### 事前確率の対数の初期化

初期値として事前確率の対数を設定します。対数を取ることで計算の安定性を保ちます。

$$ \log P(\text{spam})$$
$$ \log P(\text{ham}) $$

#### 尤度の累積計算

メール内の各単語について、スパムメールおよび非スパムメールにその単語が含まれる確率（尤度）を累積して計算します。

各単語 $w_i$ について、尤度 $P(w_i|\text{spam})$ および $P(w_i|\text{ham})$ を累積します。

$$ \log P(\text{email}|\text{spam}) = \sum_{i} \log P(w_i|\text{spam}) $$
$$ \log P(\text{email}|\text{ham}) = \sum_{i} \log P(w_i|\text{ham}) $$

##### 対数を取る理由

対数を取ることで、確率の掛け算を足し算に変えることができます。確率の積は非常に小さくなることが多いため、一般的に行われます。

$$ P(w_1|\text{spam}) \cdot P(w_2|\text{spam}) \cdot \ldots \cdot P(w_n|\text{spam}) $$
は、
$$ \log P(w_1|\text{spam}) + \log P(w_2|\text{spam}) + \ldots + \log P(w_n|\text{spam}) $$
に変換されます。

#### 事後確率の比較

累積した確率の対数を比較し、スパム確率が非スパム確率より高ければ「spam」、そうでなければ「ham」を返します。

$$ P(\text{spam}|\text{email}) > P(\text{ham}|\text{email}) \Rightarrow \text{spam} $$
$$ P(\text{spam}|\text{email}) \leq P(\text{ham}|\text{email}) \Rightarrow \text{ham} $$

この関数では、ベイズの定理に基づいて新しいメールがスパムである確率を計算し、スパムか非スパムかを予測します。

In [ ]:
# 新しいメールのスパム確率を計算する関数
def predict_spam(email):

    # 事前確率の計算
    P_spam = len(spam_emails) / len(word_count_df)
    P_ham  = len(ham_emails) / len(word_count_df)

    # 対数をとる
    # 対数の底はネイピア数 e (約 2.7) です
    log_P_spam = np.log(P_spam)
    log_P_ham  = np.log(P_ham)

    print("　スパムの事前確率", f"{np.exp(log_P_spam):.8f}", f"（対数 {log_P_spam:.4f}）")
    print("非スパムの事前確率", f"{np.exp(log_P_ham):.8f}", f"（対数 {log_P_ham:.4f}）")
    print("...")

    # ベイズの定理に従って更新する
    words = email.split()
    for word in words:
        if word in spam_word_probs:
            log_P_spam += np.log(spam_word_probs[word])
            print(f"   単語 {word} -->    スパムの確率更新 ：対数尤度 {np.log(spam_word_probs[word]):.4f} を加算")
        if word in ham_word_probs:
            log_P_ham += np.log(ham_word_probs[word])
            print(f"   単語 {word} -->  非スパムの確率更新 ：対数尤度 {np.log(ham_word_probs[word]):.4f} を加算")

    print(f"　このメールがスパムである事後確率 {np.exp(log_P_spam):.8f}", f"（対数 {log_P_spam:.4f}）")
    print(f"このメールが非スパムである事後確率 {np.exp(log_P_ham):.8f}", f"（対数 {log_P_ham:.4f}）")

    if log_P_spam > log_P_ham:
      return 'スパム'
    else:
      return '非スパム'

# テスト用のメール
test_email = 'Can we win a meeting?'
print(f'→メール "{test_email}" は {predict_spam(test_email)} と判別されました')


このコードは、Win a free prize now という新しいメールを判定しています。「win」や「prize」といった、スパムっぽい証拠がたくさん含まれているため、見事に「スパム」と判定されました。

### 📝 課題

`Can we win a meeting?` という少し奇妙なメールが届いたとします。

このメールには、「スパムっぽい単語」と「ハムっぽい単語」が混ざっています。すると、`we` と `win` の 2 単語がそれぞれ確率を更新するはずです。

（質問及び回答欄）

- `we` は、「スパムらしさ」と「ハムらしさ」のどちらをより強く上げましたか？
- `win` は、「スパムらしさ」と「ハムらしさ」のどちらをより強く上げましたか？
- 最終的な事後確率は、スパムとハムのどちらが大きいですか？

### ✍️ 総括課題

今回は、ベイズの定理という考え方を使って、AIが迷惑メールを見分ける仕組みを体験しました。教師データから確率を学習し、新しいデータに対して予測を行う、という機械学習の基本的な流れのイメージが重要です。

そこで 「教師あり学習」「教師データ」「ベイズの定理」「ナイーブベイズ」の3つの言葉をすべて使って、今回のスパムフィルターの仕組みを 2～4 行くらいで説明してください。

（回答欄）